### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [5]:
import sys
!{sys.executable} -m pip install torch

In [ ]:
import torch as t
from torch.autograd import Variable
from math import exp
import numpy

p_sat_water = 17.4725
p_sat_dio = 28.8240

x1 = numpy.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
x2 = 1 - x1
# for i in range(len(x1)):
#     x2[i] = 1 - x1[i]

p_i = numpy.array([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5])

x = Variable(t.tensor([1.0, 1]), requires_grad=True)

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable


# Define a loss
# for i in xi :
# if (1-i) in xi:
# a=i;
# b=1-i;
# p = (a * exp((x[0]*((x[1] * b)/((x[0]* a) + (x[1]*b)))) ** 2 )* p_sat_water) + (b * exp((x[1]*((x[0] * b)/((x[0]* a) + (x[1]*b)))) ** 2 ) * p_sat_dio)
a = 0.0001

for j in range(10000):
    loss1=0
    for i in range(len(x1)):
        loss = (((x1[i] * t.exp (x[0] * (((x[1] * x2[i]) / ((x[0] * x1[i])+ (x[1] * x2[i]))) ** 2))) * p_sat_water) +
                 (x2[i] * t.exp (x[1] * (((x[0] * x1[i]) / ((x[0] * x1[i]) + (x[1] * x2[i]))) ** 2)) * p_sat_dio) - p_i[i]) ** 2
        loss1= loss1+loss      
    loss1.backward()
    x.grad.numpy()
    with t.no_grad():
        x -= a * x.grad
        x.grad.zero_()

# loss = val

# Take gradient
# loss.backward()


# x.grad.numpy()

# print (loss)
print(x.data.numpy())
print(loss1.data.numpy())


In [42]:
import sys
!{sys.executable} -m pip install BayesianOptimization

In [ ]:
from bayes_opt import BayesianOptimization  


In [50]:
def func(x1,x2):
    a = (4 - 2.4 * (x1**2) + ((x1**4)/3))
    b = (-4 + 4*(x2**2))
    return ((a)*(x1**2)+(x1*x2) + (b)*(x2**2))

pbounds={'x1': (-3, 3), 'x2': (-2, 2)}

optimizer = BayesianOptimization(
    f=func,
    pbounds=pbounds,
    random_state=1
)

optimizer.maximize(
    init_points=2,
    n_iter=3,
)
print(optimizer.maximize)

|   iter    |  target   |    x1     |    x2     |
-------------------------------------------------
|  1        | -0.2834   | -0.4979   |  0.8813   |
|  2        |  85.86    | -2.999    | -0.7907   |
|  3        |  52.24    | -2.842    | -0.6774   |
|  4        |  73.11    | -2.939    | -0.9825   |
|  5        | -2.27     | -2.104    |  0.5152   |
<bound method BayesianOptimization.maximize of <bayes_opt.bayesian_optimization.BayesianOptimization object at 0x000002120743E970>>
